# Across Country Plots

Plots: 
- Latitude
  - vs crop size
  - vs activity
- Activity through the season


In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
inference_csvs = []

for root, dirs, files in os.walk(os.path.abspath(f'/gws/nopw/j04/ceh_generic/kgoldmann/')):
    for file in files:
        if file.endswith('.csv') and 'compute' not in file and 'gpu' not in file and 'inferences_tracking' in root:
            inference_csvs.append(os.path.join(root, file))


In [ ]:
len(inference_csvs)

In [ ]:
download_dir=f'./data/qc_plots/all'
plot_dir = f'./sandbox/plots/all'
os.makedirs(download_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

## Latitude Plots

In [ ]:
def create_input_df(inference_csvs):
    complete_df = pd.DataFrame()
    for c in tqdm(inference_csvs, desc='reading in the csvs'):
        try:
            input_df = pd.read_csv(c, low_memory=False)

        except Exception as e:
            print(f" - Error reading {c}: {e}")
            continue

        try:
            input_df = input_df.loc[input_df['crop_status'] != 'No detections for this image.', ]
            input_df = input_df.loc[input_df['crop_status'] != 'Image corrupt', ]

            if input_df.shape[0] == 0:
                # print(f"  - No detections in {os.path.basename(c)}")
                continue
            input_df = input_df.drop_duplicates(subset=[ 'x_min', 'x_max', 'y_min', 'y_max'])

            input_df['dep'] = os.path.basename(c).split('.')[0].split('_')[0]

            input_df['crop_area'] = (input_df['x_max'] - input_df['x_min']) * (input_df['y_max'] - input_df['y_min'])


            input_df['country'] = os.path.dirname(c).split('/')[-2].replace('_inferences_tracking', '')

            complete_df = pd.concat([complete_df, input_df[['dep',  'crop_area', 'order_name', 'country']]], ignore_index=True)
        except Exception as e:
            print(f" - Error processing {c}: {e}")
            continue
        del input_df

    complete_df = complete_df.reset_index(drop=True)
    return complete_df


In [ ]:
moth_df = create_input_df(inference_csvs)

In [ ]:
from amber_inferences.utils.config import load_credentials
from amber_inferences.utils.deployment_summary import deployment_data

In [ ]:
credentials = load_credentials("../credentials.json")

deployment_info = []

for bucket_name in moth_df['country'].unique():
    print(f"Processing {bucket_name}")
    deps = moth_df.loc[moth_df['country'] == bucket_name, 'dep'].unique()

    print(f" - Deployment: {deps}")

    # Get deployment data
    try:
        dep_data = deployment_data(
            credentials,
            subset_countries=[bucket_name.replace('rica', ' rica')],
            subset_deployments=list(deps),  # Convert to list
            include_file_count=False,
        )
        # If dep_data is a list of dictionaries, extend the list
        if isinstance(dep_data, list):
            deployment_info.extend(dep_data)
        else:
            deployment_info.append(dep_data)
    except Exception as e:
        print(f" - Error fetching deployment data for {deps}: {e}")
        continue



In [ ]:
deployment_info

In [ ]:
all_df = pd.DataFrame()
for i in range(len(deployment_info)):
    sub_dict = deployment_info[i]
    dep_df = pd.DataFrame.from_dict(sub_dict, orient='index')
    all_df = pd.concat([all_df, dep_df], ignore_index=True)

all_df = all_df.reset_index(drop=True)
all_df

In [ ]:
# match moth_df with deployment_info
moth_df = moth_df.merge(all_df, left_on='dep', how='left', right_on='deployment_id')
moth_df

In [ ]:
# subset to order_name containing 'Lepidoptera'
moth_sub = moth_df[moth_df['order_name'].str.contains('Lepidoptera', na=False)]

In [ ]:
# create a violin plot of latitude vs crop_area, subplot by country
# Set up a nice color palette
colors = sns.color_palette("Set2", n_colors=len(moth_sub['country'].unique()))
country_colors = dict(zip(sorted(moth_sub['country'].unique()), colors))

fig, axes = plt.subplots(2, 3, figsize=(18, 12), sharey=True)
fig.patch.set_facecolor('white')
axes = axes.flatten()

for i, (country, group) in enumerate(moth_sub.groupby('country')):
    ax = axes[i]

    # Create violin plot with custom styling
    parts = ax.violinplot([group[group['dep'] == dep]['crop_area'].values
                          for dep in sorted(group['dep'].unique())],
                         positions=range(len(group['dep'].unique())),
                         showmeans=True, showmedians=True)

    # Style the violin plot
    for pc in parts['bodies']:
        pc.set_facecolor(country_colors[country])
        pc.set_alpha(0.7)
        pc.set_edgecolor('darkgray')
        pc.set_linewidth(1)

    # Style the statistical lines
    parts['cmeans'].set_color('red')
    parts['cmeans'].set_linewidth(2)
    parts['cmedians'].set_color('darkblue')
    parts['cmedians'].set_linewidth(2)
    parts['cbars'].set_color('black')
    parts['cmins'].set_color('black')
    parts['cmaxes'].set_color('black')

    # Customize the axis
    ax.set_title(country.title(), fontsize=14, fontweight='bold',
                color=country_colors[country], pad=20)
    ax.set_xlabel('Deployment', fontsize=12, fontweight='bold')
    ax.set_ylabel('Crop Area (pixels²)' if i % 3 == 0 else '', fontsize=12, fontweight='bold')
    ax.set_yscale('log')

    # Set x-axis labels
    ax.set_xticks(range(len(group['dep'].unique())))
    ax.set_xticklabels(sorted(group['dep'].unique()), rotation=45, ha='right', fontsize=10)

    # Add subtle grid
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.set_facecolor('#fafafa')

    # Add border
    for spine in ax.spines.values():
        spine.set_edgecolor('gray')
        spine.set_linewidth(1.2)

# Style remaining empty plots
for ax in axes[len(moth_sub['country'].unique()):]:
    ax.set_facecolor('white')
    ax.set_xticks([])
    ax.set_xticklabels([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(False)
    for spine in ax.spines.values():
        spine.set_visible(False)

plt.tight_layout()
plt.suptitle('Crop Area Distribution of Lepidoptera by Deployment and Country',
             fontsize=18, fontweight='bold', y=0.98, color='darkslategray')

# Add a subtle background color to the figure
fig.patch.set_facecolor('#f8f9fa')

plt.savefig(os.path.join(plot_dir, 'crop_area_vs_deployment_by_country.png'),
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()


In [ ]:
def create_enhanced_deployment_plot(df, deployment_df=None, figsize=(20, 12)):
    """
    Create an enhanced violin plot using real deployment names and coordinates.

    Parameters:
    -----------
    df : pandas.DataFrame
        Dataframe with crop detection data
    deployment_df : pandas.DataFrame, optional
        DataFrame containing deployment metadata
    figsize : tuple
        Figure size (width, height)

    Returns:
    --------
    matplotlib.figure.Figure
        The created figure
    """

    # Create lookup function from deployment DataFrame
    if deployment_df is not None and not deployment_df.empty:
        print("📡 Using provided deployment DataFrame...")
        lookup_func = create_deployment_lookup_from_df(deployment_df)
    else:
        print("⚠️  No deployment DataFrame provided, using deployment IDs as names")
        def lookup_func(dep_id):
            return {
                'deployment_name': dep_id,
                'latitude': 'Unknown',
                'longitude': 'Unknown',
                'country': 'Unknown'
            }

    # Create enhanced dataframe with deployment info
    df_plot = df.copy()

    # Add deployment metadata
    print("🔍 Enhancing data with deployment information...")
    unique_deps = df_plot['dep'].unique()
    dep_info_map = {}

    for dep_id in tqdm(unique_deps, desc="Loading deployment info"):
        info = lookup_func(dep_id)
        dep_info_map[dep_id] = info

    # Add enhanced labels
    def create_enhanced_label(row):
        dep_id = row['dep']
        info = dep_info_map.get(dep_id, {})
        name = info.get('deployment_name', dep_id)
        lat = info.get('latitude', 'Unknown')
        return f"{name}\n({dep_id})\nLat: {lat}"

    df_plot['enhanced_label'] = df_plot.apply(create_enhanced_label, axis=1)
    df_plot['deployment_name'] = df_plot['dep'].map(lambda x: dep_info_map.get(x, {}).get('deployment_name', x))
    df_plot['actual_latitude'] = df_plot['dep'].map(lambda x: dep_info_map.get(x, {}).get('latitude', None))

    # Create the enhanced plot
    colors = sns.color_palette("Set2", n_colors=len(df_plot['country'].unique()))
    country_colors = dict(zip(sorted(df_plot['country'].unique()), colors))

    fig, axes = plt.subplots(2, 3, figsize=figsize, sharey=True)
    fig.patch.set_facecolor('white')
    axes = axes.flatten()

    for i, (country, group) in enumerate(df_plot.groupby('country')):
        if i >= len(axes):
            break

        ax = axes[i]

        # Create violin plot with enhanced styling
        deployments = sorted(group['dep'].unique())
        parts = ax.violinplot([group[group['dep'] == dep]['crop_area'].values
                              for dep in deployments],
                             positions=range(len(deployments)),
                             showmeans=True, showmedians=True)

        # Style the violin plot
        for pc in parts['bodies']:
            pc.set_facecolor(country_colors[country])
            pc.set_alpha(0.7)
            pc.set_edgecolor('darkgray')
            pc.set_linewidth(1)

        # Style statistical lines
        parts['cmeans'].set_color('red')
        parts['cmeans'].set_linewidth(2)
        parts['cmedians'].set_color('darkblue')
        parts['cmedians'].set_linewidth(2)
        parts['cbars'].set_color('black')
        parts['cmins'].set_color('black')
        parts['cmaxes'].set_color('black')

        # Enhanced axis customization
        ax.set_title(f'{country.title()}\n📍 Real Deployment Locations',
                    fontsize=14, fontweight='bold',
                    color=country_colors[country], pad=20)
        ax.set_xlabel('Deployment', fontsize=12, fontweight='bold')
        ax.set_ylabel('Crop Area (pixels²)' if i % 3 == 0 else '', fontsize=12, fontweight='bold')
        ax.set_yscale('log')

        # Use enhanced labels with deployment names and coordinates
        ax.set_xticks(range(len(deployments)))
        enhanced_labels = []
        for dep in deployments:
            info = dep_info_map.get(dep, {})
            name = info.get('deployment_name', dep)
            lat = info.get('latitude', 'N/A')
            # Truncate long names for readability
            display_name = name[:15] + '...' if len(name) > 15 else name
            enhanced_labels.append(f"{display_name}\n({dep})\n{lat}°N")

        ax.set_xticklabels(enhanced_labels, rotation=45, ha='right', fontsize=9)

        # Enhanced grid and styling
        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
        ax.set_facecolor('#fafafa')

        # Add border
        for spine in ax.spines.values():
            spine.set_edgecolor('darkgray')
            spine.set_linewidth(1)

    # Hide unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    plt.suptitle('🦋 Enhanced Crop Area Analysis by Deployment 🌍\nWith Real Deployment Names & Coordinates',
                 fontsize=18, fontweight='bold', y=0.98, color='darkslategray')
    plt.tight_layout()

    return fig

print("🎨 Enhanced plotting function updated to work with DataFrame!")
print("💡 Usage example:")
print("   # fig = create_enhanced_deployment_plot(moth_sub, deployment_df)")
print("   # plt.show()")

In [ ]:
# get the deployment latitudes

## Seasonal Activity Plots

In [ ]:
def activity_pop(inference_csvs):
    complete_df = pd.DataFrame()
    for c in tqdm(inference_csvs, desc='reading in the csvs'):
        try:
            input_df = pd.read_csv(c, low_memory=False)

        except Exception as e:
            print(f" - Error reading {c}: {e}")
            continue

        try:
            input_df = input_df.loc[input_df['crop_status'] != 'No detections for this image.', ]
            input_df = input_df.loc[input_df['crop_status'] != 'Image corrupt', ]

            if input_df.shape[0] == 0:
                continue

            if input_df.shape[0] == 0:
                # print(f"  - No detections in {os.path.basename(c)}")
                continue
            input_df = input_df.drop_duplicates(subset=['image_path', 'x_min', 'x_max', 'y_min', 'y_max'])


            # if deployment_name in columns set dep=deployment_name, else split on basename
            if 'deployment_name' in input_df.columns:
                input_df['dep'] = input_df['deployment_name']
            else:
                input_df['dep'] = os.path.basename(c).split('.')[0].split('_')[0]

            # get the date from the filename
            input_df['date'] = os.path.basename(c).split('.')[0].split('_')[-1]

            # get the average number of detections per depoyment per night
            input_df['avg_detections'] = input_df.groupby(['dep', 'date', 'order_name'])['image_path'].transform('count')


            input_df['country'] = os.path.dirname(c).split('/')[-2].replace('_inferences_tracking', '')
            df = input_df[['dep', 'date', 'order_name', 'country']].value_counts().reset_index()


            # # if 'latitude' in input_df.columns:
            # if 'latitude' not in input_df.columns:
            #     input_df['latitude'] = 0
            complete_df = pd.concat([complete_df, df], ignore_index=True)
        except Exception as e:
            print(f" - Error processing {c}: {e}")
            continue
        del input_df

    complete_df = complete_df.reset_index(drop=True)
    return complete_df

In [ ]:
activity_df = activity_pop(inference_csvs)

In [ ]:
# for each deployment get the average count
activity_mean = activity_df.groupby(['date', 'order_name', 'country'])['count'].mean().reset_index()

activity_mean.head()

In [ ]:
activity_mean['date'] = pd.to_datetime(activity_mean['date'], format='%Y-%m-%d')

# get the rolling average of the count
activity_mean['rolling_count_5'] = activity_mean.groupby(['order_name', 'country'])['count'].transform(lambda x: x.rolling(window=5, min_periods=1).mean())
activity_mean['rolling_count_7'] = activity_mean.groupby(['order_name', 'country'])['count'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())


# Add day of year and month for seasonal analysis
activity_mean['day_of_year'] = activity_mean['date'].dt.dayofyear

# if year = 2025, add a year to day_of_year
activity_mean.loc[activity_mean['date'].dt.year == 2025, 'day_of_year'] += 365

activity_mean['month'] = activity_mean['date'].dt.month
activity_mean['month_name'] = activity_mean['date'].dt.strftime('%b')

# Check the data
print("Date range:", activity_mean['date'].min(), "to", activity_mean['date'].max())
print("Countries:", activity_mean['country'].unique())
activity_mean.head(20)

In [ ]:
# Create seasonal activity plot - all countries on same plot with day of year as x-axis

def plot_seasonal_activity_by_order(activity_df, column='count'):
    df = activity_df.copy()

    # sort by date
    df = df.sort_values(by='date')
    df.rename(columns={'order_name': 'Order'}, inplace=True)

    # Set up beautiful color palette for countries
    countries = sorted(df['country'].unique())
    colors = sns.color_palette("husl", n_colors=len(countries))  # Vibrant, distinct colors
    country_colors = dict(zip(countries, colors))

    # Apply colors to the palette
    sns.set_palette([country_colors[country] for country in countries])

    # Create the FacetGrid using day_of_year with enhanced styling
    g = sns.FacetGrid(df, col='Order', hue='country', col_wrap=3, height=5, aspect=1.2,
                      sharey=False, sharex=False, margin_titles=True)
    g.map(sns.lineplot, 'day_of_year', column, linewidth=2.5, alpha=0.8, marker='o', markersize=4)

    # Set labels and formatting for each subplot
    for ax in g.axes.flat:
        ax.set_xlabel('')
        ax.set_ylabel('Average Crops per Night', fontsize=11, fontweight='bold')

        # Enhanced grid styling
        ax.grid(True, alpha=0.4, linestyle='-', linewidth=0.5, color='lightgray')
        ax.set_facecolor('#fafafa')  # Light background

        # Style the subplot borders
        for spine in ax.spines.values():
            spine.set_edgecolor('darkgray')
            spine.set_linewidth(1.5)

        # Enhance subplot titles
        title = ax.get_title()
        if title:
            # Extract order name and make it more readable
            order_name = title.split(' = ')[-1] if ' = ' in title else title
            ax.set_title(order_name.replace('_', ' ').title(),
                        fontsize=12, fontweight='bold', pad=15, color='darkslategray')

        # Add month labels on bottom with better styling
        month_starts = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]
        month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

        # Add second year months if data spans multiple years
        if ax.get_xlim()[1] > 365:
            month_starts_year2 = [x + 365 for x in month_starts]
            month_starts.extend(month_starts_year2)
            month_labels.extend(month_labels)

        xlim = ax.get_xlim()
        visible_months = []
        visible_labels = []

        for month_day, label in zip(month_starts, month_labels):
            if xlim[0] <= month_day <= xlim[1]:
                visible_months.append(month_day)
                visible_labels.append(label)

        if visible_months:
            ax.set_xticks(visible_months)
            ax.set_xticklabels(visible_labels, fontsize=10, fontweight='bold')
            ax.set_xlabel('Month', fontsize=11, fontweight='bold', color='darkslategray')

        # Add year labels with enhanced styling
        ax_year = ax.twiny()
        ax_year.set_frame_on(False)
        ax_year.xaxis.set_ticks_position('bottom')
        ax_year.xaxis.set_label_position('bottom')
        ax_year.spines['bottom'].set_position(('outward', 35))

        subplot_data = df
        if len(subplot_data) > 0:
            years = sorted(subplot_data['date'].dt.year.unique())
            year_positions = []
            year_labels = []

            for year in years:
                if year == years[0]:
                    year_pos = min(df['day_of_year'])
                else:
                    year_pos = 0 + (year - years[0]) * 365

                if xlim[0] <= year_pos <= xlim[1]:
                    year_positions.append(year_pos)
                    year_labels.append(str(year))

            if year_positions:
                ax_year.set_xticks(year_positions)
                ax_year.set_xticklabels(year_labels, fontsize=11, weight='bold', color='navy')
                ax_year.set_xlim(xlim)
                ax_year.set_xlabel('Year', fontsize=11, fontweight='bold', color='navy')

    # Handle insufficient data with better styling
    for ax in g.axes.flat:
        if not ax.has_data():
            ax.remove()
        else:
            order_name = ax.get_title().split(' = ')[-1] if ' = ' in ax.get_title() else ''
            order_data = df[df['Order'] == order_name] if order_name else pd.DataFrame()

            if len(order_data) < 5:
                # Style for insufficient data
                ax.set_facecolor('#f0f0f0')
                ax.text(0.5, 0.5, f'Insufficient Data\n({len(order_data)} observations)',
                       transform=ax.transAxes, ha='center', va='center',
                       fontsize=14, fontweight='bold', alpha=0.6, style='italic',
                       bbox=dict(boxstyle="round,pad=0.3", facecolor='lightcoral', alpha=0.3))

                # Hide ticks but keep the styled background
                ax.set_xticks([])
                ax.set_xticklabels([])
                ax.set_xlabel('')

                # Hide year axis for insufficient data plots
                for child_ax in ax.figure.axes:
                    if hasattr(child_ax, 'get_shared_x_axes'):
                        shared_axes = child_ax.get_shared_x_axes().get_siblings(ax)
                        if child_ax in shared_axes and child_ax != ax:
                            child_ax.set_xticks([])
                            child_ax.set_xticklabels([])
                            child_ax.set_xlabel('')

    # Enhanced legend styling
    g.add_legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left',
                borderaxespad=0., frameon=True, shadow=True, fancybox=True)

    # Style the legend
    legend = g._legend
    if legend:
        legend.set_title('Country', prop={'size': 12, 'weight': 'bold'})
        legend.get_frame().set_facecolor('white')
        legend.get_frame().set_alpha(0.9)
        legend.get_frame().set_edgecolor('darkgray')
        legend.get_frame().set_linewidth(1.5)

    plt.tight_layout()
    return g

In [ ]:
my_plt = plot_seasonal_activity_by_order(activity_mean)
plt.suptitle('🦋 Seasonal Activity Patterns: Crops per Night by Country 🦋',
             fontsize=20, fontweight='bold', y=0.98, color='darkslategray')
plt.gcf().patch.set_facecolor('#f8f9fa')  # Light background for the figure
plt.savefig(os.path.join(plot_dir, 'seasonal_activity_by_order_country.png'),
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
my_plt = plot_seasonal_activity_by_order(activity_mean, column='rolling_count_5')
plt.suptitle('🦋 Seasonal Activity Patterns: 5-Night Rolling Average 🦋\nSmoothed Trends Across Countries',
             fontsize=18, fontweight='bold', y=0.98, color='darkslategray')
plt.gcf().patch.set_facecolor('#f8f9fa')
plt.savefig(os.path.join(plot_dir, 'rolling_seasonal_activity_by_order_country_5_night.png'),
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
my_plt = plot_seasonal_activity_by_order(activity_mean, column='rolling_count_7')
plt.suptitle('🦋 Seasonal Activity Patterns: 7-Night Rolling Average 🦋\nWeekly Trends Across Countries',
             fontsize=18, fontweight='bold', y=0.98, color='darkslategray')
plt.gcf().patch.set_facecolor('#f8f9fa')
plt.savefig(os.path.join(plot_dir, 'rolling_seasonal_activity_by_order_country_7_night.png'),
            dpi=300, bbox_inches='tight', facecolor='white')
plt.show()